In [1]:
# =============================================================================
# BLOCK 1: SETUP, IMPORTS, AND DATA LOADING
# =============================================================================
import warnings
warnings.filterwarnings('ignore')
import time
import os
# --- Library Imports ---
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import catboost as cb
import optuna
from scipy.optimize import minimize
print("Libraries imported successfully.")
# --- Helper Function for Winkler Score ---
def winkler_score(y_true, lower, upper, alpha=0.1, return_coverage=False):
    width = upper - lower
    penalty_lower = np.where(y_true < lower, (2 / alpha) * (lower - y_true), 0)
    penalty_upper = np.where(y_true > upper, (2 / alpha) * (y_true - upper), 0)
    score = width + penalty_lower + penalty_upper
    if return_coverage:
        coverage = np.mean((y_true >= lower) & (y_true <= upper))
        return np.mean(score), coverage
    return np.mean(score)
# --- Global Constants ---
N_SPLITS = 5
RANDOM_STATE = 42
DATA_PATH = './'
N_OPTUNA_TRIALS = 30 # A strong number for a comprehensive search
COMPETITION_ALPHA = 0.1

# --- Load Raw Data ---
try:
    # We drop the low-variance columns they identified right away
    drop_cols=['id', 'golf', 'view_rainier', 'view_skyline', 'view_lakesamm','view_otherwater', 'view_other']
    df_train = pd.read_csv(DATA_PATH + 'dataset.csv').drop(columns=drop_cols)
    df_test = pd.read_csv(DATA_PATH + 'test.csv').drop(columns=drop_cols)
    print("Raw data loaded successfully.")
except FileNotFoundError:
    print("ERROR: Could not find 'dataset.csv' or 'test.csv'.")
    exit()
# --- Prepare Target Variable ---
y_true = df_train['sale_price'].copy()
grade_for_stratify = df_train['grade'].copy()
# The mean-error model works best when predicting the raw price directly
# So, we will NOT log-transform the target this time.
# df_train.drop('sale_price', axis=1, inplace=True) # We keep sale_price for FE
print("Setup complete.")


Libraries imported successfully.
Raw data loaded successfully.
Setup complete.


In [2]:
# Make sure to have these libraries installed
# pip install pandas numpy scikit-learn

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
import gc

# Define a random state for reproducibility
RANDOM_STATE = 42

def create_comprehensive_features(df_train, df_test):
    """
    Combines original and new advanced feature engineering steps into a single pipeline.
    """
    print("--- Starting Comprehensive Feature Engineering ---")

    # Store original indices and target variable
    train_ids = df_train.index
    test_ids = df_test.index
    y_train = df_train['sale_price'].copy() # Keep the target separate

    # Combine for consistent processing
    df_train_temp = df_train.drop(columns=['sale_price'])
    all_data = pd.concat([df_train_temp, df_test], axis=0, ignore_index=True)

    # --- Original Feature Engineering ---

    # A) Brute-Force Numerical Interactions
    print("Step 1: Creating brute-force numerical interaction features...")
    NUMS = ['area', 'land_val', 'imp_val', 'sqft_lot', 'sqft', 'sqft_1', 'grade', 'year_built']
    # Ensure all columns exist and are numeric, fill missing with 0 for safety
    for col in NUMS:
        if col not in all_data.columns:
            all_data[col] = 0
        else:
            all_data[col] = pd.to_numeric(all_data[col], errors='coerce').fillna(0)
            
    for i in range(len(NUMS)):
        for j in range(i + 1, len(NUMS)):
            all_data[f'{NUMS[i]}_x_{NUMS[j]}'] = all_data[NUMS[i]] * all_data[NUMS[j]]

    # B) Date Features
    print("Step 2: Creating date features...")
    all_data['sale_date'] = pd.to_datetime(all_data['sale_date'])
    all_data['sale_year'] = all_data['sale_date'].dt.year
    all_data['sale_month'] = all_data['sale_date'].dt.month
    all_data['sale_dayofyear'] = all_data['sale_date'].dt.dayofyear
    all_data['age_at_sale'] = all_data['sale_year'] - all_data['year_built']

    # C) TF-IDF Text Features
    print("Step 3: Creating TF-IDF features for text columns...")
    text_cols = ['subdivision', 'zoning', 'city', 'sale_warning', 'join_status', 'submarket']
    all_data[text_cols] = all_data[text_cols].fillna('missing').astype(str)
    
    for col in text_cols:
        tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3, 5), max_features=128, binary=True)
        svd = TruncatedSVD(n_components=8, random_state=RANDOM_STATE)
        
        tfidf_matrix = tfidf.fit_transform(all_data[col])
        tfidf_svd = svd.fit_transform(tfidf_matrix)
        
        tfidf_df = pd.DataFrame(tfidf_svd, columns=[f'{col}_tfidf_svd_{i}' for i in range(8)])
        all_data = pd.concat([all_data, tfidf_df], axis=1)

    # D) Log transform some interaction features
    for c in ['land_val_x_imp_val', 'land_val_x_sqft', 'imp_val_x_sqft']:
        if c in all_data.columns:
            all_data[c] = np.log1p(all_data[c].fillna(0))

    # --- New Feature Engineering Ideas ---

    # F) Group-By Aggregation Features
    print("Step 4: Creating group-by aggregation features...")
    group_cols = ['submarket', 'city', 'zoning']
    num_cols_for_agg = ['grade', 'sqft', 'imp_val', 'land_val', 'age_at_sale']

    for group_col in group_cols:
        for num_col in num_cols_for_agg:
            agg_stats = all_data.groupby(group_col)[num_col].agg(['mean', 'std', 'max', 'min']).reset_index()
            agg_stats.columns = [group_col] + [f'{group_col}_{num_col}_{stat}' for stat in ['mean', 'std', 'max', 'min']]
            all_data = pd.merge(all_data, agg_stats, on=group_col, how='left')
            all_data[f'{num_col}_minus_{group_col}_mean'] = all_data[num_col] - all_data[f'{group_col}_{num_col}_mean']

    # G) Ratio Features
    print("Step 5: Creating ratio features...")
    # Add a small epsilon to prevent division by zero
    epsilon = 1e-6 
    all_data['total_val'] = all_data['imp_val'] + all_data['land_val']
    all_data['imp_val_to_land_val_ratio'] = all_data['imp_val'] / (all_data['land_val'] + epsilon)
    all_data['land_val_ratio'] = all_data['land_val'] / (all_data['total_val'] + epsilon)
    all_data['sqft_to_lot_ratio'] = all_data['sqft'] / (all_data['sqft_lot'] + epsilon)
    all_data['was_renovated'] = (all_data['year_reno'] > 0).astype(int)
    all_data['reno_age_at_sale'] = np.where(all_data['was_renovated'] == 1, all_data['sale_year'] - all_data['year_reno'], -1)

    # H) Geospatial Clustering Features
    print("Step 6: Creating geospatial clustering features...")
    coords = all_data[['latitude', 'longitude']].copy()
    coords.fillna(coords.median(), inplace=True) # Simple imputation

    # KMeans is sensitive to feature scaling, but for lat/lon it's often okay without it.
    kmeans = KMeans(n_clusters=20, random_state=RANDOM_STATE, n_init=10) 
    all_data['location_cluster'] = kmeans.fit_predict(coords)
    
    # Calculate distance to each cluster center
    cluster_centers = kmeans.cluster_centers_
    for i in range(len(cluster_centers)):
        center = cluster_centers[i]
        all_data[f'dist_to_cluster_{i}'] = np.sqrt((coords['latitude'] - center[0])**2 + (coords['longitude'] - center[1])**2)

    # --- Final Cleanup ---
    print("Step 7: Finalizing feature set...")
    cols_to_drop = ['sale_date', 'subdivision', 'zoning', 'city', 'sale_warning', 'join_status', 'submarket']
    all_data = all_data.drop(columns=cols_to_drop)

    # One-hot encode the new cluster feature
    all_data = pd.get_dummies(all_data, columns=['location_cluster'], prefix='loc_cluster')
    
    # Final check for any remaining object columns to be safe (besides index)
    object_cols = all_data.select_dtypes(include='object').columns
    if len(object_cols) > 0:
        print(f"Warning: Found unexpected object columns: {object_cols}. Dropping them.")
        all_data = all_data.drop(columns=object_cols)
        
    all_data.fillna(0, inplace=True)

    # Separate back into train and test sets
    train_len = len(train_ids)
    X = all_data.iloc[:train_len].copy()
    X_test = all_data.iloc[train_len:].copy()
    
    # Restore original indices
    X.index = train_ids
    X_test.index = test_ids
    
    # Align columns - crucial for model prediction
    X_test = X_test[X.columns]
    
    print(f"\nComprehensive FE complete. Total features: {X.shape[1]}")
    gc.collect()
    
    return X, X_test, y_train
# =============================================================================
# BLOCK 2.5: EXECUTE FEATURE ENGINEERING
# =============================================================================
print("\n--- Starting Block 2.5: Executing Feature Engineering Pipeline ---")

# This is the crucial step that was missing.
# We call the function to create our training and testing dataframes.
X, X_test, y_train = create_comprehensive_features(df_train, df_test)

# Let's verify the output
print(f"Feature engineering complete. X shape: {X.shape}, X_test shape: {X_test.shape}")
gc.collect()


--- Starting Block 2.5: Executing Feature Engineering Pipeline ---
--- Starting Comprehensive Feature Engineering ---
Step 1: Creating brute-force numerical interaction features...
Step 2: Creating date features...
Step 3: Creating TF-IDF features for text columns...
Step 4: Creating group-by aggregation features...
Step 5: Creating ratio features...
Step 6: Creating geospatial clustering features...
Step 7: Finalizing feature set...

Comprehensive FE complete. Total features: 233
Feature engineering complete. X shape: (200000, 233), X_test shape: (200000, 233)


0

In [3]:
# =============================================================================
# BLOCK 3: LOAD ALL PRE-TRAINED MODEL PREDICTIONS
# =============================================================================

# Define paths to your saved prediction files
PREDS_SAVE_PATH = './mean_models_v1/' # For XGB and CatBoost preds
NN_PREDS_PATH = './NN_model_predictions/' # For NN preds

print("--- Loading all base model predictions from saved .npy files... ---")
try:
    # Load Mean Model OOF (Out-of-Fold) Predictions
    oof_xgb_preds = np.load(os.path.join(PREDS_SAVE_PATH, 'oof_xgb_preds.npy'))
    oof_cb_preds = np.load(os.path.join(PREDS_SAVE_PATH, 'oof_cb_preds.npy'))
    oof_lgbm_preds = np.load(os.path.join(PREDS_SAVE_PATH, 'oof_lgbm_preds.npy'))
    oof_nn_preds = np.load(os.path.join(NN_PREDS_PATH, 'oof_nn_preds.npy'))
    
    # Load Mean Model Test Predictions
    test_xgb_preds = np.load(os.path.join(PREDS_SAVE_PATH, 'test_xgb_preds.npy'))
    test_cb_preds = np.load(os.path.join(PREDS_SAVE_PATH, 'test_cb_preds.npy'))
    test_lgbm_preds = np.load(os.path.join(PREDS_SAVE_PATH, 'test_lgbm_preds.npy'))
    test_nn_preds = np.load(os.path.join(NN_PREDS_PATH, 'test_nn_preds.npy'))
    
    print("All MEAN model predictions loaded successfully.")
    
except FileNotFoundError as e:
    print(f"\nERROR: Could not find a required prediction file. {e}")
    print("Please ensure you have run all training notebooks and saved their predictions first.")

--- Loading all base model predictions from saved .npy files... ---
All MEAN model predictions loaded successfully.


In [4]:
# =============================================================================
# BLOCK 4: BUILD AND EVALUATE THE STAGE 1 (MEAN) ENSEMBLE
# =============================================================================

print("\n--- Finding optimal weights for the 4-model mean ensemble ---")

# Stack the OOF predictions for easy matrix multiplication
oof_preds_stack_mean = np.vstack([oof_xgb_preds, oof_cb_preds, oof_lgbm_preds, oof_nn_preds]).T

# Define the function for the optimizer to minimize (RMSE)
def get_ensemble_rmse(weights):
    final_prediction = np.dot(oof_preds_stack_mean, weights)
    return np.sqrt(mean_squared_error(y_true, final_prediction))

# Run the optimizer to find the best weights
# Constraints ensure weights sum to 1 and are between 0 and 1
result = minimize(get_ensemble_rmse, [1/4]*4, method='SLSQP', bounds=[(0,1)]*4, 
                  constraints=({'type': 'eq', 'fun': lambda w: 1 - np.sum(w)}))
best_mean_weights = result.x

# Create the final blended mean predictions for both OOF and test sets
oof_ensemble_mean = np.dot(oof_preds_stack_mean, best_mean_weights)
test_ensemble_mean = np.dot(np.vstack([test_xgb_preds, test_cb_preds, test_lgbm_preds, test_nn_preds]).T, best_mean_weights)

print("\n--- STAGE 1 (MEAN) ENSEMBLE RESULTS ---")
print(f"Optimal Weights (XGB/CB/LGBM/NN): {best_mean_weights[0]:.4f} / {best_mean_weights[1]:.4f} / {best_mean_weights[2]:.4f} / {best_mean_weights[3]:.4f}")
print(f"Final Ensemble Mean OOF RMSE: ${np.sqrt(mean_squared_error(y_true, oof_ensemble_mean)):,.2f}")


--- Finding optimal weights for the 4-model mean ensemble ---

--- STAGE 1 (MEAN) ENSEMBLE RESULTS ---
Optimal Weights (XGB/CB/LGBM/NN): 0.2107 / 0.4508 / 0.2485 / 0.0900
Final Ensemble Mean OOF RMSE: $94,960.11


In [5]:
# =============================================================================
# BLOCK 5: TUNE AND TRAIN THE XGBOOST ERROR MODEL
# =============================================================================

# --- 1. Prepare Features and Target for the Error Model ---
print("\n--- Preparing features for the XGBoost error model ---")
# The error target is the absolute difference between the blended mean and the true price
error_target = np.abs(y_true - oof_ensemble_mean)

# The features are the full feature set PLUS the OOF predictions of the mean models
X_for_error = X.copy()
X_for_error['oof_xgb'] = oof_xgb_preds
X_for_error['oof_cb'] = oof_cb_preds
X_for_error['oof_lgbm'] = oof_lgbm_preds
X_for_error['oof_nn'] = oof_nn_preds

# Do the same for the test set, ensuring column names are consistent
X_test_for_error = X_test.copy()
X_test_for_error['oof_xgb'] = test_xgb_preds
X_test_for_error['oof_cb'] = test_cb_preds
X_test_for_error['oof_lgbm'] = test_lgbm_preds
X_test_for_error['oof_nn'] = test_nn_preds

# --- 2. Tune the Error Model with Optuna ---
print("\n--- Tuning the XGBoost Error Model with Optuna... ---")
X_train_err_opt, X_val_err_opt, y_train_err_opt, y_val_err_opt = train_test_split(
    X_for_error, error_target, test_size=0.2, random_state=RANDOM_STATE
)
dtrain_err_opt = xgb.DMatrix(X_train_err_opt, label=y_train_err_opt)
dval_err_opt = xgb.DMatrix(X_val_err_opt, label=y_val_err_opt)

def objective_error_model(trial):
    params = {
        'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist',
        'n_jobs': -1, 'seed': RANDOM_STATE,
        'eta': trial.suggest_float('eta', 0.01, 0.05, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 6),
        'subsample': trial.suggest_float('subsample', 0.6, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
        'lambda': trial.suggest_float('lambda', 1.0, 50.0, log=True),
        'alpha': trial.suggest_float('alpha', 1.0, 50.0, log=True)
    }
    bst = xgb.train(params, dtrain_err_opt, num_boost_round=2000, 
                    evals=[(dval_err_opt, 'val')], early_stopping_rounds=75, verbose_eval=False)
    preds = bst.predict(dval_err_opt, iteration_range=(0, bst.best_iteration))
    return np.sqrt(mean_squared_error(y_val_err_opt, preds))

study_error = optuna.create_study(direction='minimize')
study_error.optimize(objective_error_model, n_trials=70)
best_params_error = study_error.best_params
best_params_error['n_estimators'] = study_error.best_trial.user_attrs.get('best_iteration', 2000)
print(f"\nError model tuning complete. Best validation RMSE: ${study_error.best_value:,.2f}")

# --- 3. Final K-Fold Training of the Error Model ---
print("\n--- K-Fold training the final error model... ---")
oof_error_preds = np.zeros(len(X))
test_error_preds = np.zeros(len(X_test))
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_for_error, grade_for_stratify)):
    print(f"  Training XGB error model for fold {fold+1}/{N_SPLITS}...")
    X_train_err, X_val_err = X_for_error.iloc[train_idx], X_for_error.iloc[val_idx]
    y_train_err, y_val_err = error_target.iloc[train_idx], error_target.iloc[val_idx]

    model_err = xgb.XGBRegressor(**best_params_error)
    model_err.fit(X_train_err, y_train_err)
    
    oof_error_preds[val_idx] = np.clip(model_err.predict(X_val_err), 0, None)
    test_error_preds += np.clip(model_err.predict(X_test_for_error), 0, None) / N_SPLITS

print(f"\nFinal Error Model OOF RMSE: ${np.sqrt(mean_squared_error(error_target, oof_error_preds)):,.2f}")


--- Preparing features for the XGBoost error model ---

--- Tuning the XGBoost Error Model with Optuna... ---


[I 2025-07-24 22:48:17,859] A new study created in memory with name: no-name-85cb1abc-f49b-4bb8-bd85-9e9c693d1cd6
[I 2025-07-24 22:48:27,967] Trial 0 finished with value: 61569.60421908874 and parameters: {'eta': 0.02596274385884213, 'max_depth': 5, 'subsample': 0.7165035200223228, 'colsample_bytree': 0.7553808604507141, 'lambda': 1.3621040017388202, 'alpha': 4.855224057249528}. Best is trial 0 with value: 61569.60421908874.
[I 2025-07-24 22:48:42,827] Trial 1 finished with value: 61176.599508171304 and parameters: {'eta': 0.04596427351611171, 'max_depth': 6, 'subsample': 0.7755357269088934, 'colsample_bytree': 0.6040542831570711, 'lambda': 25.4559921097289, 'alpha': 10.431141936315996}. Best is trial 1 with value: 61176.599508171304.
[I 2025-07-24 22:49:19,446] Trial 2 finished with value: 61160.35264782224 and parameters: {'eta': 0.013854342741406706, 'max_depth': 6, 'subsample': 0.8806135554312897, 'colsample_bytree': 0.8585148328772866, 'lambda': 35.58153391734345, 'alpha': 9.58098


Error model tuning complete. Best validation RMSE: $61,114.44

--- K-Fold training the final error model... ---
  Training XGB error model for fold 1/5...
  Training XGB error model for fold 2/5...
  Training XGB error model for fold 3/5...
  Training XGB error model for fold 4/5...
  Training XGB error model for fold 5/5...

Final Error Model OOF RMSE: $60,484.58


In [6]:
# =============================================================================
# BLOCK 6: FINAL CALIBRATION AND SUBMISSION
# =============================================================================

print("\n--- Calibrating the final intervals with an Optimizer ---")
oof_error_final = np.clip(oof_error_preds, 0, None)

def get_winkler_from_multipliers(multipliers):
    a, b = multipliers[0], multipliers[1]
    low = oof_ensemble_mean - oof_error_final * a
    high = oof_ensemble_mean + oof_error_final * b
    return winkler_score(y_true, low, high)

initial_guess = [1.5, 1.5]
bounds = [(0.5, 4.0), (0.5, 4.0)]
result_calib = minimize(get_winkler_from_multipliers, initial_guess, method='L-BFGS-B', bounds=bounds)
best_a, best_b = result_calib.x
best_score = result_calib.fun

print("\n" + "="*60); print("FINAL RESULTS"); print("="*60)
print(f"Final OOF Winkler Score (4 Mean + 1 XGB Error): ${best_score:,.2f}")
print(f"Optimal Multipliers: a={best_a:.4f}, b={best_b:.4f}")




--- Calibrating the final intervals with an Optimizer ---

FINAL RESULTS
Final OOF Winkler Score (4 Mean + 1 XGB Error): $293,131.26
Optimal Multipliers: a=1.9797, b=2.1747


In [ ]:
# --- Create Submission File ---
print("\n--- Creating final submission file... ---")
final_lower = test_ensemble_mean - np.clip(test_error_preds, 0, None) * best_a
final_upper = test_ensemble_mean + np.clip(test_error_preds, 0, None) * best_b
final_upper = np.maximum(final_lower + 1, final_upper) # Safety check

submission_df = pd.DataFrame({'id': pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))['id'], 
                              'pi_lower': final_lower, 
                              'pi_upper': final_upper})
submission_filename = f'submission_3M1E_XGB_{int(best_score)}.csv'
submission_df.to_csv(submission_filename, index=False)
print(f"\n'{submission_filename}' created successfully!")
display(submission_df.head())

In [7]:
# =============================================================================
# BLOCK 5 (CATBOOST): TUNE AND TRAIN THE CATBOOST ERROR MODEL
# =============================================================================

# --- 1. Prepare Features and Target for the Error Model ---
print("\n--- Preparing features for the CatBoost error model ---")
# The error target is the same for all error models
# The features are also the same (full feature set + OOF mean preds)

# --- 2. Tune the Error Model with Optuna ---
print("\n--- Tuning the CatBoost Error Model with Optuna... ---")
# We can use the same train/validation split for tuning efficiency
# X_train_err_opt, X_val_err_opt, y_train_err_opt, y_val_err_opt are already defined from the XGBoost block

def objective_error_model_cb(trial):
    params = {
        'loss_function': 'RMSE',
        'eval_metric': 'RMSE',
        'random_seed': RANDOM_STATE,
        'verbose': 0,
        'iterations': trial.suggest_int('iterations', 1000, 3000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05, log=True),
        'depth': trial.suggest_int('depth', 3, 6),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 50.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 0.9),
        'random_strength': trial.suggest_float('random_strength', 1.0, 10.0, log=True)
    }
    
    model = cb.CatBoostRegressor(**params)
    model.fit(
        X_train_err_opt, y_train_err_opt,
        eval_set=[(X_val_err_opt, y_val_err_opt)],
        early_stopping_rounds=75,
        use_best_model=True
    )
    
    preds = model.predict(X_val_err_opt)
    rmse = np.sqrt(mean_squared_error(y_val_err_opt, preds))
    return rmse

study_error_cb = optuna.create_study(direction='minimize')
study_error_cb.optimize(objective_error_model_cb, n_trials=70)
best_params_error_cb = study_error_cb.best_params
print(f"\nCatBoost Error model tuning complete. Best validation RMSE: ${study_error_cb.best_value:,.2f}")

# --- 3. Final K-Fold Training of the Error Model ---
print("\n--- K-Fold training the final CatBoost error model... ---")
oof_error_preds_cb = np.zeros(len(X))
test_error_preds_cb = np.zeros(len(X_test))
skf_cb = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

for fold, (train_idx, val_idx) in enumerate(skf_cb.split(X_for_error, grade_for_stratify)):
    print(f"  Training CatBoost error model for fold {fold+1}/{N_SPLITS}...")
    X_train_err, X_val_err = X_for_error.iloc[train_idx], X_for_error.iloc[val_idx]
    y_train_err, y_val_err = error_target.iloc[train_idx], error_target.iloc[val_idx]

    model_err_cb = cb.CatBoostRegressor(**best_params_error_cb, early_stopping_rounds=75, verbose=0)
    model_err_cb.fit(X_train_err, y_train_err, eval_set=[(X_val_err, y_val_err)])
    
    oof_error_preds_cb[val_idx] = np.clip(model_err_cb.predict(X_val_err), 0, None)
    test_error_preds_cb += np.clip(model_err_cb.predict(X_test_for_error), 0, None) / N_SPLITS

print(f"\nFinal CatBoost Error Model OOF RMSE: ${np.sqrt(mean_squared_error(error_target, oof_error_preds_cb)):,.2f}")

[I 2025-07-24 23:09:37,443] A new study created in memory with name: no-name-ecfa461e-e897-430a-ac90-1a25ee78589f



--- Preparing features for the CatBoost error model ---

--- Tuning the CatBoost Error Model with Optuna... ---


[I 2025-07-24 23:09:54,845] Trial 0 finished with value: 61469.03336599517 and parameters: {'iterations': 2482, 'learning_rate': 0.019112171719833218, 'depth': 6, 'l2_leaf_reg': 5.678502599472514, 'subsample': 0.6367525143282068, 'random_strength': 5.478896729634377}. Best is trial 0 with value: 61469.03336599517.
[I 2025-07-24 23:10:04,739] Trial 1 finished with value: 61959.377937245226 and parameters: {'iterations': 2071, 'learning_rate': 0.016222624946220735, 'depth': 4, 'l2_leaf_reg': 1.0917476416508312, 'subsample': 0.6107510045067848, 'random_strength': 2.917492128292178}. Best is trial 0 with value: 61469.03336599517.
[I 2025-07-24 23:10:20,910] Trial 2 finished with value: 61523.53355184494 and parameters: {'iterations': 2412, 'learning_rate': 0.01689411019542091, 'depth': 6, 'l2_leaf_reg': 1.6872331844834152, 'subsample': 0.6030289135677183, 'random_strength': 2.719275110251027}. Best is trial 0 with value: 61469.03336599517.
[I 2025-07-24 23:10:27,992] Trial 3 finished with 


CatBoost Error model tuning complete. Best validation RMSE: $61,323.22

--- K-Fold training the final CatBoost error model... ---
  Training CatBoost error model for fold 1/5...
  Training CatBoost error model for fold 2/5...
  Training CatBoost error model for fold 3/5...
  Training CatBoost error model for fold 4/5...
  Training CatBoost error model for fold 5/5...

Final CatBoost Error Model OOF RMSE: $60,550.52


In [11]:
# =============================================================================
# BLOCK 5.5 (LIGHTGBM): TUNE AND TRAIN THE LIGHTGBM ERROR MODEL
# =============================================================================
import lightgbm as lgb

# --- 1. Prepare Features and Target for the Error Model ---
print("\n--- Preparing features for the LightGBM error model ---")
# The error target and features are the same as the other error models.

# --- 2. Tune the Error Model with Optuna ---
print("\n--- Tuning the LightGBM Error Model with Optuna... ---")
# We use the same train/validation split for tuning efficiency.
# X_train_err_opt, X_val_err_opt, y_train_err_opt, y_val_err_opt are already defined.

def objective_error_model_lgbm(trial):
    params = {
        'objective': 'regression_l1', # MAE is robust for error modeling
        'metric': 'rmse',
        'random_state': RANDOM_STATE,
        'n_jobs': -1,
        'boosting_type': 'gbdt',
        'verbosity': -1, # Suppress warnings
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 1000, 3000),
        'num_leaves': trial.suggest_int('num_leaves', 10, 40), # Shallower for error model
        'max_depth': trial.suggest_int('max_depth', 3, 6),
        'lambda_l1': trial.suggest_float('lambda_l1', 1.0, 50.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1.0, 50.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 0.9),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 0.9),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7)
    }
    
    model = lgb.LGBMRegressor(**params)
    model.fit(
        X_train_err_opt, y_train_err_opt,
        eval_set=[(X_val_err_opt, y_val_err_opt)],
        callbacks=[lgb.early_stopping(75, verbose=False)]
    )
    
    preds = model.predict(X_val_err_opt)
    rmse = np.sqrt(mean_squared_error(y_val_err_opt, preds))
    return rmse

study_error_lgbm = optuna.create_study(direction='minimize')
study_error_lgbm.optimize(objective_error_model_lgbm, n_trials=30) # 30 trials is a good balance
best_params_error_lgbm = study_error_lgbm.best_params
print(f"\nLightGBM Error model tuning complete. Best validation RMSE: ${study_error_lgbm.best_value:,.2f}")

# --- 3. Final K-Fold Training of the Error Model ---
print("\n--- K-Fold training the final LightGBM error model... ---")
oof_error_preds_lgbm = np.zeros(len(X))
test_error_preds_lgbm = np.zeros(len(X_test))
skf_lgbm = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

for fold, (train_idx, val_idx) in enumerate(skf_lgbm.split(X_for_error, grade_for_stratify)):
    print(f"  Training LightGBM error model for fold {fold+1}/{N_SPLITS}...")
    X_train_err, X_val_err = X_for_error.iloc[train_idx], X_for_error.iloc[val_idx]
    y_train_err, y_val_err = error_target.iloc[train_idx], error_target.iloc[val_idx]

    model_err_lgbm = lgb.LGBMRegressor(**best_params_error_lgbm, early_stopping_rounds=75)
    model_err_lgbm.fit(X_train_err, y_train_err,
                       eval_set=[(X_val_err, y_val_err)],
                       callbacks=[lgb.early_stopping(75, verbose=False)])
    
    oof_error_preds_lgbm[val_idx] = np.clip(model_err_lgbm.predict(X_val_err), 0, None)
    test_error_preds_lgbm += np.clip(model_err_lgbm.predict(X_test_for_error), 0, None) / N_SPLITS

print(f"\nFinal LightGBM Error Model OOF RMSE: ${np.sqrt(mean_squared_error(error_target, oof_error_preds_lgbm)):,.2f}")

[I 2025-07-24 17:14:55,327] A new study created in memory with name: no-name-ba92639c-e5cd-4056-9d88-270b2c0882ee



--- Preparing features for the LightGBM error model ---

--- Tuning the LightGBM Error Model with Optuna... ---


[I 2025-07-24 17:15:12,321] Trial 0 finished with value: 63094.69733961664 and parameters: {'learning_rate': 0.034793142258152235, 'n_estimators': 2339, 'num_leaves': 37, 'max_depth': 5, 'lambda_l1': 2.4131530218075854, 'lambda_l2': 7.969999565684957, 'feature_fraction': 0.7255296573441212, 'bagging_fraction': 0.8051794666918468, 'bagging_freq': 2}. Best is trial 0 with value: 63094.69733961664.
[I 2025-07-24 17:15:32,145] Trial 1 finished with value: 63243.20911394577 and parameters: {'learning_rate': 0.030072630780943625, 'n_estimators': 2389, 'num_leaves': 31, 'max_depth': 6, 'lambda_l1': 14.575271598553988, 'lambda_l2': 8.70101780320695, 'feature_fraction': 0.7608235405531482, 'bagging_fraction': 0.7987340416954959, 'bagging_freq': 4}. Best is trial 0 with value: 63094.69733961664.
[I 2025-07-24 17:15:52,071] Trial 2 finished with value: 63157.0332211606 and parameters: {'learning_rate': 0.025100431663170962, 'n_estimators': 2711, 'num_leaves': 35, 'max_depth': 5, 'lambda_l1': 5.16


LightGBM Error model tuning complete. Best validation RMSE: $62,960.37

--- K-Fold training the final LightGBM error model... ---
  Training LightGBM error model for fold 1/5...
Training until validation scores don't improve for 75 rounds
  Training LightGBM error model for fold 2/5...
Training until validation scores don't improve for 75 rounds
Early stopping, best iteration is:
[471]	valid_0's l2: 3.61934e+09
  Training LightGBM error model for fold 3/5...
Training until validation scores don't improve for 75 rounds
  Training LightGBM error model for fold 4/5...
Training until validation scores don't improve for 75 rounds
  Training LightGBM error model for fold 5/5...
Training until validation scores don't improve for 75 rounds
Early stopping, best iteration is:
[651]	valid_0's l2: 3.6831e+09

Final LightGBM Error Model OOF RMSE: $60,583.05


In [8]:
# =============================================================================
# BLOCK 7: SAVE ALL SPECIALIST ERROR MODEL PREDICTIONS
# =============================================================================
import os
import numpy as np

# --- Define a dedicated path for error model predictions ---
ERROR_PREDS_PATH = './error_models/' 
os.makedirs(ERROR_PREDS_PATH, exist_ok=True) # Create the directory if it doesn't exist

print(f"--- Saving all error model prediction arrays to '{ERROR_PREDS_PATH}' ---")

# --- Save XGBoost Error Model Predictions ---
# This assumes your variables are named 'oof_error_preds' and 'test_error_preds'
try:
    np.save(os.path.join(ERROR_PREDS_PATH, 'oof_error_preds_xgb.npy'), oof_error_preds)
    np.save(os.path.join(ERROR_PREDS_PATH, 'test_error_preds_xgb.npy'), test_error_preds)
    print("XGBoost error predictions saved successfully.")
except NameError:
    print("ERROR: Could not find XGBoost error prediction variables ('oof_error_preds', 'test_error_preds'). Please ensure the XGBoost error model block was run.")

# --- Save CatBoost Error Model Predictions ---
# This assumes your variables are named 'oof_error_preds_cb' and 'test_error_preds_cb'
try:
    np.save(os.path.join(ERROR_PREDS_PATH, 'oof_error_preds_cb.npy'), oof_error_preds_cb)
    np.save(os.path.join(ERROR_PREDS_PATH, 'test_error_preds_cb.npy'), test_error_preds_cb)
    print("CatBoost error predictions saved successfully.")
except NameError:
    print("ERROR: Could not find CatBoost error prediction variables ('oof_error_preds_cb', 'test_error_preds_cb'). Please ensure the CatBoost error model block was run.")

try:
    np.save(os.path.join(ERROR_PREDS_PATH, 'oof_error_preds_lgbm.npy'), oof_error_preds)
    np.save(os.path.join(ERROR_PREDS_PATH, 'test_error_preds_lgbm.npy'), test_error_preds)
    print("LightGBM error predictions saved successfully.")
except NameError:
    print("ERROR: Could not find LightGBM error prediction variables ('oof_error_preds', 'test_error_preds'). Please ensure the LightGBM error model block was run.")


print("\nAll necessary prediction files are now ready for the final blending notebook.")

--- Saving all error model prediction arrays to './error_models/' ---
XGBoost error predictions saved successfully.
CatBoost error predictions saved successfully.
LightGBM error predictions saved successfully.

All necessary prediction files are now ready for the final blending notebook.


In [9]:
# =======================================================================================================
# FINAL BLOCK (DEFINITIVE): OPTIMAL ERROR BLEND, PRECISE CALIBRATION, AND SUBMISSION
# =======================================================================================================
from scipy.optimize import minimize
import pandas as pd
import numpy as np
import os

# --- 1. Search for the Optimal Blend of Error Models ---
print("\n--- Searching for the optimal blend weight for the two error models... ---")

best_blend_score = float('inf')
best_blend_weight = 0.5 # Default to 50/50
best_blend_a = 1.0
best_blend_b = 1.0

# We will test weights for the XGBoost error model from 0% to 100% in 5% increments
for xgb_weight in np.arange(0, 1.05, 0.05):
    cb_weight = 1.0 - xgb_weight
    
    # Create the blended error predictions for this specific weight
    oof_error_blend = (oof_error_preds * xgb_weight) + (oof_error_preds_cb * cb_weight)
    oof_error_final_ensemble = np.clip(oof_error_blend, 0, None)

    # --- Calibrate THIS specific blend ---
    def get_winkler_from_multipliers(multipliers):
        a, b = multipliers[0], multipliers[1]
        low = oof_ensemble_mean - oof_error_final_ensemble * a
        high = oof_ensemble_mean + oof_error_final_ensemble * b
        return winkler_score(y_true, low, high)

    initial_guess = [1.9, 2.2]
    bounds = [(1.0, 3.0), (1.0, 3.0)]
    result_calib = minimize(get_winkler_from_multipliers, initial_guess, method='L-BFGS-B', bounds=bounds)
    
    current_score = result_calib.fun
    current_a, current_b = result_calib.x
    
    print(f"  Testing Blend (XGB/CB): {xgb_weight:.2f}/{cb_weight:.2f}  |  Score: ${current_score:,.2f} (a={current_a:.3f}, b={current_b:.3f})")

    # If this blend is the best so far, save its parameters
    if current_score < best_blend_score:
        best_blend_score = current_score
        best_blend_weight = xgb_weight
        best_blend_a = current_a
        best_blend_b = current_b

# --- 2. Build the Final Error Ensemble with the Best Weight ---
print("\n--- Building the final error ensemble with the optimal blend... ---")
final_xgb_weight = best_blend_weight
final_cb_weight = 1.0 - final_xgb_weight
test_error_final_ensemble = (test_error_preds * final_xgb_weight) + (test_error_preds_cb * final_cb_weight)

# --- 3. Final Results and Submission ---
print("\n" + "="*60); print("FINAL RESULTS & SUBMISSION"); print("="*60)
print(f"Final OOF Winkler Score (Optimally Blended Ensemble of Ensembles): ${best_blend_score:,.2f}")
print(f"Optimal Error Blend (XGB/CB): {final_xgb_weight:.2f} / {final_cb_weight:.2f}")
print(f"Optimal Multipliers: a={best_blend_a:.4f}, b={best_blend_b:.4f}")

# Create the final prediction intervals for the test set
final_lower = test_ensemble_mean - np.clip(test_error_final_ensemble, 0, None) * best_blend_a
final_upper = test_ensemble_mean + np.clip(test_error_final_ensemble, 0, None) * best_blend_b
final_upper = np.maximum(final_lower + 1, final_upper)




--- Searching for the optimal blend weight for the two error models... ---
  Testing Blend (XGB/CB): 0.00/1.00  |  Score: $294,441.63 (a=1.977, b=2.172)
  Testing Blend (XGB/CB): 0.05/0.95  |  Score: $294,103.50 (a=1.977, b=2.173)
  Testing Blend (XGB/CB): 0.10/0.90  |  Score: $293,791.04 (a=1.977, b=2.175)
  Testing Blend (XGB/CB): 0.15/0.85  |  Score: $293,510.02 (a=1.976, b=2.174)
  Testing Blend (XGB/CB): 0.20/0.80  |  Score: $293,253.40 (a=1.978, b=2.175)
  Testing Blend (XGB/CB): 0.25/0.75  |  Score: $293,025.20 (a=1.978, b=2.175)
  Testing Blend (XGB/CB): 0.30/0.70  |  Score: $292,824.74 (a=1.977, b=2.175)
  Testing Blend (XGB/CB): 0.35/0.65  |  Score: $292,649.75 (a=1.979, b=2.177)
  Testing Blend (XGB/CB): 0.40/0.60  |  Score: $292,503.35 (a=1.978, b=2.178)
  Testing Blend (XGB/CB): 0.45/0.55  |  Score: $292,390.86 (a=1.979, b=2.176)
  Testing Blend (XGB/CB): 0.50/0.50  |  Score: $292,307.99 (a=1.979, b=2.176)
  Testing Blend (XGB/CB): 0.55/0.45  |  Score: $292,251.11 (a=1.98

In [10]:
# Create the submission DataFrame
submission_df = pd.DataFrame({'id': pd.read_csv('./test.csv')['id'], 
                              'pi_lower': final_lower, 
                              'pi_upper': final_upper})

# Save the submission file
submission_filename = f'submission_final_OptimalEoE_{int(best_blend_score)}.csv'
submission_df.to_csv(submission_filename, index=False)

print(f"\n'{submission_filename}' created successfully! Good luck on the leaderboard!")
display(submission_df.head())


'submission_final_OptimalEoE_292680.csv' created successfully! Good luck on the leaderboard!


,id,pi_lower,pi_upper
0,200000,818976.003200,1.012106e+06
1,200001,578449.575873,7.983806e+05
2,200002,450278.368874,6.537181e+05
3,200003,294652.612316,4.240232e+05
4,200004,346969.921670,7.985549e+05


In [11]:
er_PATH = './error_models'
oof_error_preds_lgbm = np.load(os.path.join(er_PATH, 'oof_error_preds_lgbm.npy'))
test_error_preds_lgbm = np.load(os.path.join(er_PATH, 'test_error_preds_lgbm.npy'))


In [12]:
# =======================================================================================================
# FINAL BLOCK (DEFINITIVE): OPTIMAL 3-ERROR-MODEL BLEND, PRECISE CALIBRATION, AND SUBMISSION
# =======================================================================================================
from scipy.optimize import minimize
import pandas as pd
import numpy as np
import os

# --- 1. Define the Objective Function for the Optimizer ---
# This master function will find the best blend AND the best calibration simultaneously.

print("\n--- Searching for the optimal blend of 3 error models AND their calibration multipliers... ---")

# Stack the OOF error predictions for easy matrix multiplication
oof_error_stack = np.vstack([oof_error_preds, oof_error_preds_cb, oof_error_preds_lgbm]).T

def get_winkler_from_blend_and_calibration(params):
    """
    This function takes a single array of parameters and unpacks it:
    - params[0]: weight for XGB error model
    - params[1]: weight for CatBoost error model
    - params[2]: weight for LightGBM error model
    - params[3]: multiplier 'a' for the lower bound
    - params[4]: multiplier 'b' for the upper bound
    """
    # Unpack the parameters
    weights = params[:3]
    a = params[3]
    b = params[4]
    
    # Create the blended error prediction using the given weights
    oof_error_blend = np.dot(oof_error_stack, weights)
    oof_error_final_ensemble = np.clip(oof_error_blend, 0, None)
    
    # Calculate the calibrated prediction intervals
    low = oof_ensemble_mean - oof_error_final_ensemble * a
    high = oof_ensemble_mean + oof_error_final_ensemble * b
    
    # Return the Winkler score for this specific combination
    return winkler_score(y_true, low, high)

# --- 2. Run the Optimizer ---
# We need an initial guess and bounds for all 5 parameters (3 weights + 2 multipliers)
initial_guess = [1/3, 1/3, 1/3, 1.9, 2.2]
bounds = [(0, 1), (0, 1), (0, 1), (1.0, 3.0), (1.0, 3.0)]

# Add a constraint that the sum of the weights must equal 1
weights_constraint = {'type': 'eq', 'fun': lambda params: 1.0 - np.sum(params[:3])}

result = minimize(
    fun=get_winkler_from_blend_and_calibration,
    x0=initial_guess,
    bounds=bounds,
    constraints=[weights_constraint],
    method='SLSQP'
)

# Extract the best parameters found by the optimizer
best_params = result.x
best_score = result.fun
best_error_weights = best_params[:3]
best_a = best_params[3]
best_b = best_params[4]

# --- 3. Build the Final Test Set Predictions ---
print("\n--- Building the final test set predictions with the optimal blend... ---")
test_error_stack = np.vstack([test_error_preds, test_error_preds_cb, test_error_preds_lgbm]).T
test_error_final_ensemble = np.dot(test_error_stack, best_error_weights)

# --- 4. Final Results and Submission ---
print("\n" + "="*60); print("FINAL RESULTS & SUBMISSION"); print("="*60)
print(f"Final OOF Winkler Score (4 Mean + 3 Error Ensemble): ${best_score:,.2f}")
print(f"Optimal Error Blend (XGB/CB/LGBM): {best_error_weights[0]:.4f} / {best_error_weights[1]:.4f} / {best_error_weights[2]:.4f}")
print(f"Optimal Multipliers: a={best_a:.4f}, b={best_b:.4f}")




--- Searching for the optimal blend of 3 error models AND their calibration multipliers... ---

--- Building the final test set predictions with the optimal blend... ---

FINAL RESULTS & SUBMISSION
Final OOF Winkler Score (4 Mean + 3 Error Ensemble): $292,217.24
Optimal Error Blend (XGB/CB/LGBM): 0.3107 / 0.3786 / 0.3107
Optimal Multipliers: a=1.9799, b=2.1764


In [13]:
# Create the final prediction intervals for the test set
final_lower = test_ensemble_mean - np.clip(test_error_final_ensemble, 0, None) * best_a
final_upper = test_ensemble_mean + np.clip(test_error_final_ensemble, 0, None) * best_b
final_upper = np.maximum(final_lower + 1, final_upper)

# Create the submission DataFrame
submission_df = pd.DataFrame({'id': pd.read_csv('./test.csv')['id'], 
                              'pi_lower': final_lower, 
                              'pi_upper': final_upper})

# Save the submission file
submission_filename = f'submission_final_4M3E_{int(best_score)}.csv'
submission_df.to_csv(submission_filename, index=False)

print(f"\n'{submission_filename}' created successfully! Good luck on the leaderboard!")
display(submission_df.head())


'submission_final_4M3E_292217.csv' created successfully! Good luck on the leaderboard!


,id,pi_lower,pi_upper
0,200000,820615.127941,1.010704e+06
1,200001,579589.665406,7.975820e+05
2,200002,452163.714092,6.520662e+05
3,200003,293769.395373,4.252615e+05
4,200004,354575.178634,7.911283e+05
